In [ ]:
import pandas as pd

#import the transport csv

#the is Data for the whole thing
transport = pd.read_csv('Data/wu03ew_msoa.csv')


bus_loc = pd.read_csv('Data/Bus_locations.csv')
train_loc = pd.read_csv('Data/Train locations.csv')
metro_loc = pd.read_csv('Data/Metro_locations.csv')

In [ ]:
#merge by origin MSOA and sum over all transport options
transport_msoa = transport.groupby("Area of residence", as_index=False).sum()
transport_msoa

In [ ]:

bus_loc = bus_loc.rename(columns = {'Freq': 'Bus_stops'})
bus_loc

In [ ]:
train_loc = train_loc.rename(columns = {'Freq': 'Train_stations'})

In [ ]:
metro_loc = metro_loc.rename(columns = {'Freq': 'metro_stations'})

In [ ]:
transport_msoa1 = pd.merge(transport_msoa,
                          bus_loc,
                           left_on = "Area of residence",
                           right_on = "Var1",
                          how = 'left')

In [ ]:
transport_msoa1.isna().sum()

In [ ]:
transport_msoa2 = pd.merge(transport_msoa1,
                          train_loc,
                          left_on = "Area of residence",
                          right_on = "Var1",
                          how = 'left')

In [ ]:
transport_msoa3 = pd.merge(transport_msoa2,
                          metro_loc,
                          left_on = "Area of residence",
                          right_on = "Var1",
                          how = 'left')

In [ ]:
transport_msoa3.isna().sum()

In [ ]:
#transport_msoa3.to_csv("Data/transport_msoa.csv", index=False)

In [ ]:
transport_msoa3

In [ ]:
households = pd.read_csv("Data/MSOA households.csv")
cars = pd.read_csv("Data/household cars.csv")

In [ ]:
households

In [ ]:
cars

In [ ]:
household_cars= pd.merge(households,
                          cars,
                           left_on = "MSOA Code",
                         right_on = "geography code",
                          how = 'left')

In [ ]:
pd.set_option('display.max_columns', None)
household_cars

In [ ]:
household_cars["HHs_owning cars"] = household_cars["Cars: All categories: Car or van availability; measures: Value"] - household_cars["Cars: No cars or vans in household; measures: Value"]

In [ ]:
household_cars["HH_owning_cars_perc"] = household_cars["HHs_owning cars"] / household_cars["Cars: All categories: Car or van availability; measures: Value"]  

In [ ]:
household_cars

In [ ]:
#merging on household cars owned
transport_msoa4 = pd.merge(transport_msoa3,
                          household_cars[["HH_owning_cars_perc", "MSOA Code"]],
                          left_on = "Area of residence",
                          right_on = "MSOA Code",
                          how = 'left')

In [ ]:
#filling all NaNs with 0
transport_msoa4.fillna(0)

In [ ]:
#writing to the csv
#transport_msoa4.to_csv("Data/transport_msoa2.csv", index=False)

In [ ]:
#reading in the csv written to before
transport = pd.read_csv("Data/transport_msoa2.csv")

In [ ]:
#changing all modes of transport to percentages
transport["work_from_home_perc"] = transport["Work mainly at or from home"] / transport["All categories: Method of travel to work"] *100
transport["underground_metro_perc"] = transport["Underground, metro, light rail, tram"] / transport["All categories: Method of travel to work"] *100
transport["train_perc"] = transport["Train"] / transport["All categories: Method of travel to work"] *100
transport["bus_perc"] = transport["Bus, minibus or coach"] / transport["All categories: Method of travel to work"] *100
transport["taxi_perc"] = transport["Taxi"] / transport["All categories: Method of travel to work"] *100
transport["motorcycle_perc"] = transport["Motorcycle, scooter or moped"] / transport["All categories: Method of travel to work"] *100
transport["car_or_van_drive_perc"] = transport["Driving a car or van"] / transport["All categories: Method of travel to work"] *100
transport["car_or_van_passenger_perc"] = transport["Passenger in a car or van"] / transport["All categories: Method of travel to work"] *100
transport["bicycle_perc"] = transport["Bicycle"] / transport["All categories: Method of travel to work"] *100
transport["on_foot_perc"] = transport["On foot"] / transport["All categories: Method of travel to work"] *100
transport["other_perc"] = transport["Other method of travel to work"] / transport["All categories: Method of travel to work"] *100

In [ ]:
#checking that the transport percentage has worked
transport

In [ ]:
#reading in the travel times
travel_car = pd.read_csv("Data/travel_time_car.csv")
travel_bus = pd.read_csv("Data/travel_time_bus.csv")
travel_rail = pd.read_csv("Data/travel_time_rail.csv")

In [ ]:

#merging transport with travel by car time
transport = pd.merge(transport,
                    travel_car,
                    left_on = "Area of residence",
                    right_on = "origin_msoacode",
                    how = 'left')

In [ ]:
#merging with travel by bus times
transport = pd.merge(transport,
                    travel_bus,
                    left_on = "Area of residence",
                    right_on = "origin_msoacode",
                    how = 'left')

In [ ]:
#merging with travel by rail times
transport = pd.merge(transport,
                    travel_rail,
                    left_on = "Area of residence",
                    right_on = "origin_msoacode",
                    how = 'left')

In [ ]:
transport.to_csv("Data/transport_msoa3.csv", index=False)

In [ ]:
transport

In [ ]:
#transport.isna().sum()

In [ ]:
# Create a new clean dataframe

# Remove all flow census counts
transport_final = transport.drop(transport.columns[1:13], axis=1)

# remove other columns by name
transport_final.drop(['Var1_x', 'Var1_y', 'Var1', 'MSOA Code', 'origin_msoacode_x', 'car', 'cumulative_time_car',
                     'origin_msoacode_y', 'Bus, minibus or coach_y', 'cumulative_time_bus', 'origin_msoacode',
                     'Underground, metro, light rail, tram_y', 'cumulative_time_rail'], axis=1, inplace=True)
transport_final

In [ ]:
# merge car owners with car passengers
transport_final['car_perc'] = transport_final['car_or_van_drive_perc'] + transport_final['car_or_van_passenger_perc']
# remove the old columns
transport_final.drop(['car_or_van_drive_perc', 'car_or_van_passenger_perc'], axis=1, inplace=True)

In [ ]:
# change car ownership to % (just need to multiply by 100)
transport_final['HH_owning_cars_perc'] = transport_final['HH_owning_cars_perc'] * 100
transport_final

In [ ]:
# rename first column
transport_final.rename({'Area of residence': 'MSOA'}, axis=1, inplace=True)

In [ ]:
# save as csv
transport_final.to_csv("Data/cleaned_transport.csv", index=False)